In [ ]:
import pandas as pd
import json
import pycountry_convert as pc
%load_ext vis_examples

# Support for Current CCT VIS Workflow in Hatchet

We begin by loading real data, downloaded from Kaggle, into a pandas dataframe and loading derived information from that dataframe.

In [ ]:
dataset = pd.read_csv('Examples/Electricity_Production_By_Source.csv')
vis_in = dataset.groupby(['Entity']).max().reset_index()


def getContinent(row):
    try:
        c_code = pc.country_name_to_country_alpha2(row['Entity'], cn_name_format='default')
        return pc.country_alpha2_to_continent_code(c_code)
    except:
        return 'OTH'

vis_in['Continent'] = vis_in.apply(getContinent, axis=1)

In [ ]:
vis_in

We can use the basic functionality in roundtrip to pass this data into our scatterplot as a DataFrame (or other complex datatype). The visualization developer -- whoever built scatterplot -- will manage the necessary conversions.

In [ ]:
%scatter_plt vis_in

Once visualized, we can return our filter selections back to our notebook with a magic function provided by the visualization developer `get_filter`. This function makes a pass through call to a "return" function provided by our Roundtrip API and stores the data in `fltr`.

In [ ]:
%get_filter fltr

In [ ]:
fltr

`fltr` Is returned from the "get_filter" function as a jsonified string so we can load it and then filter our visualization. If we re-run our above `scatter_plt` cell we will see that the selected circles are now gone.

In [ ]:
fltr = json.loads(fltr)
vis_in = vis_in[~vis_in.index.isin(fltr)]

## More Complex and Dynamic Workflows with `?` operator

### Automatic passing of javascript updatesbetween multiple views

We load data like we did before.

In [ ]:
dataset = pd.read_csv('Examples/Electricity_Production_By_Source.csv')
vis_in = dataset.groupby(['Entity']).max().reset_index()


def getContinent(row):
    try:
        c_code = pc.country_name_to_country_alpha2(row['Entity'], cn_name_format='default')
        return pc.country_alpha2_to_continent_code(c_code)
    except:
        return 'OTH'

vis_in['Continent'] = vis_in.apply(getContinent, axis=1)

In the following cell you will see that the visualization developer provided an optional second argument to our scatterplot `out`. You will also see that this argument has a new operator `?`.

The `?` operator is provided by roundtrip to allow users the freedom to define Jupyter variables that they would like to act like 'passed by refrence' variables and auto-update when changed elsewhere in the notebook.

A visualization developer may disable this two way binding if they so choose. In the `scatter_plt` is it optional for both our variables `vis_in` and `out`.

In [ ]:
%scatter_plt vis_in ?out

Because `out` is bound with the `?` operator, selections made in the above plot will be automatically reflected in the following cell after it has been run at least once.

In [ ]:
%show_table ?out

In [ ]:
%scatter_plt ?out ?out2

In [ ]:
%show_table ?out2

### Automatic updating of RT visualizations on jupyter variables change

Here we define our dataset once again and visualize it.

In [ ]:
dataset = pd.read_csv('Examples/Electricity_Production_By_Source.csv')
climate_data = dataset.groupby(['Entity']).max().reset_index()


def getContinent(row):
    try:
        c_code = pc.country_name_to_country_alpha2(row['Entity'], cn_name_format='default')
        return pc.country_alpha2_to_continent_code(c_code)
    except:
        return 'OTH'

climate_data['Continent'] = climate_data.apply(getContinent, axis=1)

In [ ]:
%scatter_plt ?climate_data ?selection

In [ ]:
%show_table ?selection

To get rid of some of our "garbage" data programatically, we can filter out all the data assocaited with the `oth` continent. Because we have specified `climate_data` to auto-update above we will see the `scatter_plt` reload with our new `climate_data`.

In [ ]:
climate_data = climate_data[climate_data['Continent'] != 'OTH']

We can also do other interesting things like filter to only get data from Europe. These manipulations are not supported by the visualization but enabled through the magic of Roundtrip.

In [ ]:
climate_data = climate_data[climate_data['Continent'] == 'EU']